# 와인데이터 크롤링해오기 

In [12]:
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import math

In [2]:
driver = webdriver.Chrome('./driver/chromedriver')

In [29]:
url = 'https://www.wine21.com/13_search/wine_list.html'
driver.get(url)

In [30]:
# 평점순으로 (변경되기까지 시간이 오래걸림)
xpath = '//*[@id="shOrder1"]/option[4]'
driver.find_element(By.XPATH,xpath).click()

- 첫번째 게시글 클릭 후 
- 상품명, 도수, 생산국가/지역, 품종, 추천음식, 음용온도  (상세정보 크롤링)

In [102]:
# 첫번째 와인 클릭 
driver.find_element(By.XPATH,'//*[@id="wine_list"]/li[1]/div[2]/div[1]/h3/a').click()
time.sleep(1)

경우1: 가격정보 없는경우 (용량때문에 가져오는것 필요없으면 삭제할 것)
경우2: 상세정보에 추천음식이 있는 경우 

In [100]:
# 상세정보 가져오기 
wine_category=driver.find_element(By.XPATH, '/html/body/section/div[3]/div[2]/div[2]/div[1]/p/span[1]').text
wine_name = driver.find_element(By.CLASS_NAME,'wine-name').text
wine_flavor = driver.find_element(By.XPATH,'/html/body/section/div[3]/div[2]/div[2]/div[2]').text
# flavor a 태그 class="on"으로 들어가있기때문에 값이 나오지않음 ..
wine_food = driver.find_elements(By.XPATH,'/html/body/section/div[3]/div[2]/div[2]/div[3]/div[2]/div[2]/ul/li/p')
# wind_food가 리스트로 들어감 -> 리스트 안에 리스트가 있어버림 
# 데이터프레임으로 만들때 중첩리스트!! 기억할 것 
wine_capacity = driver.find_element(By.CSS_SELECTOR,'body > section > div.inner > div.clear > div.wine-top-right > p.wine-price > strong').text
wine_text = driver.find_element(By.CLASS_NAME,'view-tab-inner').text
wine = [wine_category,wine_name, wine_food, wine_capacity,wine_text]

# 전체 목록으로 돌아가기 
driver.back() 

In [101]:
wine

['화이트',
 '제라르 베르트랑, 아트 비브르 블랑',
 [<selenium.webdriver.remote.webelement.WebElement (session="dc7be6508c35ebeab32915e485cc633e", element="3A75335D7902257A422A16C92C8A5CAE_element_3578")>,
  <selenium.webdriver.remote.webelement.WebElement (session="dc7be6508c35ebeab32915e485cc633e", element="3A75335D7902257A422A16C92C8A5CAE_element_3574")>,
  <selenium.webdriver.remote.webelement.WebElement (session="dc7be6508c35ebeab32915e485cc633e", element="3A75335D7902257A422A16C92C8A5CAE_element_3579")>,
  <selenium.webdriver.remote.webelement.WebElement (session="dc7be6508c35ebeab32915e485cc633e", element="3A75335D7902257A422A16C92C8A5CAE_element_3580")>,
  <selenium.webdriver.remote.webelement.WebElement (session="dc7be6508c35ebeab32915e485cc633e", element="3A75335D7902257A422A16C92C8A5CAE_element_3581")>,
  <selenium.webdriver.remote.webelement.WebElement (session="dc7be6508c35ebeab32915e485cc633e", element="3A75335D7902257A422A16C92C8A5CAE_element_3582")>],
 '가격정보없음',
 '생산자\n제라르 베르트랑 Gerard Ber

In [80]:
driver.find_element(By.CSS_SELECTOR,'body > section > div.inner > div.clear > div.wine-top-right > p.wine-price > strong').text
# 용량만 가져오고 싶은데 가격도 가져와짐 

'가격정보없음'

In [95]:
driver.find_element(By.XPATH,'/html/body/section/div[3]/div[2]/div[2]/div[2]').text

'당도\n산도\n바디\n타닌'

In [103]:
driver.find_element(By.XPATH,'//*[@id="wine_list"]/li['+i+']/div[2]/div[1]/h3/a').click()
time.sleep(1)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.110)


In [75]:
# 블로그 참고한 주석 
for index in range(13719, 13249):
    url=f'https://www.wine21.com/13_search/wine_view.html?Idx={index}'
    req = requests.get(url)
    if req.status_code != requests.codes.ok:
        print('접속 실패')
        continue
    html = BeaturifulSoup(req.text,'html.parser')
    names = html.select(".cnt")
    names_list = [name.text for name in names]
    print(names_list)

In [16]:
# 와인 페이지 더보기 버튼 끝까지 클릭
# 28,474개의 와인으로 시간이 오래걸림 (보류)
while driver.find_element(By.CSS_SELECTOR, '#wineListMoreBtn').is_displayed:
    try:
        WebDriverWait(driver,1).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#wineListMoreBtn"))
        ).click()
    except TimeoutException:
        break

NameError: name 'TimeoutException' is not defined

In [ ]:
# 끝까지 내려간 후 
# https://www.wine21.com/13_search/wine_view.html?Idx=174581&lq=LIST 
# 숫자 확인해서 세부내용 크롤링할 for문 작성해볼것! 